In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
from qtcodes import TopologicalRegister, TopologicalCircuit, REPETITION, XXZZ, XZZX
from qiskit import execute, Aer, QuantumCircuit, QuantumRegister, ClassicalRegister

# Helper

In [113]:
def add_xxxx(qc, syndrome, ancilla, qs):
    top_l, top_r, bot_l, bot_r = qs
    qc.h(syndrome)
    if top_r is not None:
        qc.cx(syndrome, top_r)
    if top_l is not None:
        qc.cx(syndrome, top_l)
    if bot_r is not None:
        qc.cx(syndrome, bot_r)
    if bot_l is not None:
        qc.cx(syndrome, bot_l)
    qc.h(syndrome)
    qc.measure(syndrome, ancilla)
    qc.barrier()
    
def add_zzzz(qc, syndrome, ancilla, qs):
    top_l, top_r, bot_l, bot_r = qs
    if top_r is not None:
        qc.cx(top_r, syndrome)
    if bot_r is not None:
        qc.cx(bot_r, syndrome)
    if top_l is not None:
        qc.cx(top_l, syndrome)
    if bot_l is not None:
        qc.cx(bot_l, syndrome)
    qc.measure(syndrome, ancilla)
    qc.barrier()
    

In [173]:
from qiskit import execute, Aer
def sim(circ):
    return execute(circ, 
                  Aer.get_backend("aer_simulator"), 
                  shots=10,
                 ).result().get_counts()

# Protected ZZ

In [177]:
treg = TopologicalRegister(ctypes=[XXZZ, XXZZ], params=[{"d": 3}, {"d":3}])
tcirc = TopologicalCircuit(treg)
tcirc.reset_z(0)
tcirc.reset_z(1)

tcirc.x(0)
tcirc.x(1)

data_0 = tcirc[0].lattice.qregisters["data"]
data_1 = tcirc[1].lattice.qregisters["data"]


num_r = 2
qr = QuantumRegister(num_r,"qr_parity")
cr = ClassicalRegister(num_r,"cr_parity")
tcirc.circ.add_register(qr)
tcirc.circ.add_register(cr)

# ZZ
add_zzzz(tcirc.circ, qr[0], cr[0], (data_0[6],data_0[7],data_1[0],None))
add_zzzz(tcirc.circ, qr[1], cr[1], (None,data_0[8],data_1[1],data_1[2]))

In [178]:
results = sim(tcirc.circ)
results

{'00': 10}

# Protected XX

In [189]:
treg = TopologicalRegister(ctypes=[XXZZ, XXZZ], params=[{"d": 3}, {"d":3}])
tcirc = TopologicalCircuit(treg)
tcirc.reset_x(0)
tcirc.reset_x(1)

# tcirc.z(0)
# tcirc.z(1)

data_0 = tcirc[0].lattice.qregisters["data"]
data_1 = tcirc[1].lattice.qregisters["data"]


num_r = 2
qr = QuantumRegister(num_r,"qr_parity")
cr = ClassicalRegister(num_r,"cr_parity")
tcirc.circ.add_register(qr)
tcirc.circ.add_register(cr)


# XX
add_xxxx(tcirc.circ, qr[0], cr[0], (data_0[2],data_1[0],None,data_1[3]))
add_xxxx(tcirc.circ, qr[1], cr[1], (data_0[5],None,data_0[8],data_1[6]))

In [190]:
results = sim(tcirc.circ)
results

{'00': 10}